In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import tensorflow as tf
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.autograd import Variable 

In [44]:
df=pd.read_csv('../input/fraud-detection/fraudTrain.csv')
df.isna().sum()

Unnamed: 0               0
trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
unix_time                0
merch_lat                0
merch_long               0
is_fraud                 0
dtype: int64

In [45]:
df

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296670,1296670,2020-06-21 12:12:08,30263540414123,fraud_Reichel Inc,entertainment,15.56,Erik,Patterson,M,162 Jessica Row Apt. 072,...,37.7175,-112.4777,258,Geoscientist,1961-11-24,440b587732da4dc1a6395aba5fb41669,1371816728,36.841266,-111.690765,0
1296671,1296671,2020-06-21 12:12:19,6011149206456997,fraud_Abernathy and Sons,food_dining,51.70,Jeffrey,White,M,8617 Holmes Terrace Suite 651,...,39.2667,-77.5101,100,"Production assistant, television",1979-12-11,278000d2e0d2277d1de2f890067dcc0a,1371816739,38.906881,-78.246528,0
1296672,1296672,2020-06-21 12:12:32,3514865930894695,fraud_Stiedemann Ltd,food_dining,105.93,Christopher,Castaneda,M,1632 Cohen Drive Suite 639,...,32.9396,-105.8189,899,Naval architect,1967-08-30,483f52fe67fabef353d552c1e662974c,1371816752,33.619513,-105.130529,0
1296673,1296673,2020-06-21 12:13:36,2720012583106919,"fraud_Reinger, Weissnat and Strosin",food_dining,74.90,Joseph,Murray,M,42933 Ryan Underpass,...,43.3526,-102.5411,1126,Volunteer coordinator,1980-08-18,d667cdcbadaaed3da3f4020e83591c83,1371816816,42.788940,-103.241160,0


In [46]:
df['age']=dt.date.today().year-pd.to_datetime(df['dob']).dt.year

df_fraud=df['is_fraud']
df_select=df[['category','amt','lat','long','city_pop','age']]

In [47]:
df_select

,category,amt,lat,long,city_pop,age
0,misc_net,4.97,36.0788,-81.1781,3495,34
1,grocery_pos,107.23,48.8878,-118.2105,149,44
2,entertainment,220.11,42.1808,-112.2620,4154,60
3,gas_transport,45.00,46.2306,-112.1138,1939,55
4,misc_pos,41.96,38.4207,-79.4629,99,36
...,...,...,...,...,...,...
1296670,entertainment,15.56,37.7175,-112.4777,258,61
1296671,food_dining,51.70,39.2667,-77.5101,100,43
1296672,food_dining,105.93,32.9396,-105.8189,899,55
1296673,food_dining,74.90,43.3526,-102.5411,1126,42


In [48]:
df_select.category.unique()

array(['misc_net', 'grocery_pos', 'entertainment', 'gas_transport',
       'misc_pos', 'grocery_net', 'shopping_net', 'shopping_pos',
       'food_dining', 'personal_care', 'health_fitness', 'travel',
       'kids_pets', 'home'], dtype=object)

In [49]:
df_category=df_select.category

In [50]:
df_category

0               misc_net
1            grocery_pos
2          entertainment
3          gas_transport
4               misc_pos
               ...      
1296670    entertainment
1296671      food_dining
1296672      food_dining
1296673      food_dining
1296674      food_dining
Name: category, Length: 1296675, dtype: object

In [51]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_category)
features = vectorizer.get_feature_names()

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [52]:
type(features)

list

In [53]:
category_vector=X.toarray()

In [54]:
df_select=df_select.drop(['category'],axis=1)

In [55]:
cate=pd.DataFrame(category_vector, columns = features)

In [56]:
fianl_dataset = pd.concat([df_select, cate], axis=1)

In [57]:
fianl_dataset

,amt,lat,long,city_pop,age,entertainment,food_dining,gas_transport,grocery_net,grocery_pos,health_fitness,home,kids_pets,misc_net,misc_pos,personal_care,shopping_net,shopping_pos,travel
0,4.97,36.0788,-81.1781,3495,34,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,107.23,48.8878,-118.2105,149,44,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,220.11,42.1808,-112.2620,4154,60,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,45.00,46.2306,-112.1138,1939,55,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,41.96,38.4207,-79.4629,99,36,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296670,15.56,37.7175,-112.4777,258,61,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1296671,51.70,39.2667,-77.5101,100,43,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1296672,105.93,32.9396,-105.8189,899,55,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1296673,74.90,43.3526,-102.5411,1126,42,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [58]:
features=np.array(fianl_dataset)
label=np.array(df_fraud)


In [59]:
from imblearn.over_sampling import SMOTE
from collections import Counter
from matplotlib import pyplot
from numpy import where
oversample = SMOTE()
X_r, y = oversample.fit_resample(features, label)
# summarize the new class distribution
counter = Counter(y)
print(counter)
# scatter plot of examples by class label
for label, _ in counter.items():
    row_ix = where(y == label)[0]

Counter({0: 1289169, 1: 1289169})


In [60]:
from sklearn.preprocessing import StandardScaler

X_r2 = StandardScaler().fit_transform(X_r)

In [61]:
X_r2.shape

(2578338, 19)

In [62]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X_r2, y, test_size=0.3)
X_train_tensors = Variable(torch.Tensor(X_train))
X_test_tensors = Variable(torch.Tensor(X_test))

y_train_tensors = Variable(torch.Tensor(y_train))
y_test_tensors = Variable(torch.Tensor(y_test)) 

In [64]:
X_train_tensors_final = torch.reshape(X_train_tensors,   (X_train_tensors.shape[0],1, X_train_tensors.shape[1]))

y_train_tensors_final = torch.reshape(y_train_tensors,   (y_train_tensors.shape[0],1))

X_test_tensors_final = torch.reshape(X_test_tensors,  (X_test_tensors.shape[0],1, X_test_tensors.shape[1])) 
y_test_tensors_final = torch.reshape(y_test_tensors,   (y_test_tensors.shape[0],1))

In [ ]:
X_train_tensors_final

In [ ]:
print("Training Shape", X_train_tensors_final.shape, y_train_tensors_final.shape)
print("Testing Shape", X_test_tensors_final.shape, y_test_tensors.shape) 


mean = np.mean(train_features, axis=0)
train_features -= mean
val_features -= mean
std = np.std(train_features, axis=0)
train_features /= std
val_features /= std

In [65]:
device=torch.device('cuda')
def get_lstm_params(vocab_size, num_hiddens, device):
    num_inputs = num_outputs = vocab_size

    def normal(shape):
        return torch.randn(size=shape, device=device)*0.01

    def three():
        return (normal((num_inputs, num_hiddens)),
                normal((num_hiddens, num_hiddens)),
                torch.zeros(num_hiddens, device=device))

    W_xi, W_hi, b_i = three()  
    W_xf, W_hf, b_f = three()  
    W_xo, W_ho, b_o = three()  
    W_xc, W_hc, b_c = three()  

    W_hq = normal((num_hiddens, num_outputs))
    b_q = torch.zeros(num_outputs, device=device)

    params = [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc,
              b_c, W_hq, b_q]
    for param in params:
        param.requires_grad_(True)
    return params

def init_lstm_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device),
            torch.zeros((batch_size, num_hiddens), device=device))

def lstm(inputs, state, params):
    [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c,
     W_hq, b_q] = params
    (H, C) = state
    outputs = []
    for X in inputs:
        I = torch.sigmoid((X @ W_xi) + (H @ W_hi) + b_i)
        F = torch.sigmoid((X @ W_xf) + (H @ W_hf) + b_f)
        O = torch.sigmoid((X @ W_xo) + (H @ W_ho) + b_o)
        C_tilda = torch.tanh((X @ W_xc) + (H @ W_hc) + b_c)
        C = F * C + I * C_tilda
        H = O * torch.tanh(C)
        Y = (H @ W_hq) + b_q
        outputs.append(Y)
    return torch.cat(outputs, dim=0), (H, C)



In [66]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class LSTM1(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM1, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) 
        self.fc_1 =  nn.Linear(hidden_size, 128) 
        self.fc = nn.Linear(128, num_classes) 

        #self.sigm = nn.Sigmoid()
        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size,device=x.device)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size,device=x.device)) #internal state
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0))
        
        hn = hn.view(-1, self.hidden_size) 
        out = self.relu(hn)
        out = self.fc_1(out) 
        out = self.relu(out) 
        out = self.fc(out) 
        
        return out


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MyModel(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(MyModel, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True)
        self.label = nn.Linear(hidden_size, num_classes)


    def forward(self,x):


        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size,device=x.device))
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size,device=x.device))


        output, (final_hidden_state, final_cell_state) = self.lstm(x, (h_0, c_0))
        final_hidden_state = final_hidden_state.view(-1, self.hidden_size)

        return self.label(final_hidden_state) 

In [67]:
num_epochs = 30 
learning_rate = 0.001

input_size = 19 
hidden_size = 5 
num_layers = 1 

num_classes = 1  
model = LSTM1(num_classes, input_size, hidden_size, num_layers, X_train_tensors_final.shape[1]) #our lstm class

In [68]:
criterion = torch.nn.MSELoss().cuda() if torch.cuda.is_available() else torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

outputs = lstm1.forward(X_train_tensors_final) #forward pass
optimizer.zero_grad() #caluclate the gradient, manually setting to 0

# obtain the loss function
loss = criterion(outputs, y_train_tensors_final)

loss.backward() #calculates the loss of the loss function

optimizer.step() #improve from loss, i.e backprop


In [ ]:
y_train_tensors_final.shape

In [ ]:
print(X_train_tensors_final.shape)
print(y_train_tensors_final.shape)
X_test_tensors_final
y_test_tensors_final

In [ ]:

outputs=model(X_train_tensors_final)
loss = criterion(outputs, y_train_tensors_final)

In [ ]:
loss

from tqdm import tqdm
optimizer.zero_grad()
   

In [69]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for epoch in range(num_epochs):
  outputs = model(X_train_tensors_final).to(device)
 #forward pass
  optimizer.zero_grad() #caluclate the gradient, manually setting to 0
 
  # obtain the loss function
  loss = criterion(outputs, y_train_tensors_final.to(device))
 
  loss.backward() #calculates the loss of the loss function
 
  optimizer.step() #improve from loss, i.e backprop
  if epoch % 5 == 0:
    #scores = outputs.evaluate(X_test_tensors, Y_test_tensors, verbose=0)
    print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 

Epoch: 0, loss: 0.43959
Epoch: 5, loss: 0.36609
Epoch: 10, loss: 0.31139
Epoch: 15, loss: 0.27524
Epoch: 20, loss: 0.25532
Epoch: 25, loss: 0.24704


In [77]:
torch.save(model.state_dict(), "/kaggle/working/w.pth")

In [81]:

# Load the model that we saved at the end of the training loop 
model = LSTM1(num_classes, input_size, hidden_size, num_layers, X_train_tensors_final.shape[1])
path = "w.pth" 
model.load_state_dict(torch.load(path)) 

running_accuracy = 0 
total = 0 

model
    

LSTM1(
  (lstm): LSTM(19, 5, batch_first=True)
  (fc_1): Linear(in_features=5, out_features=128, bias=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
)

In [85]:
y_test_tensors_final

tensor([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]])

In [86]:
m = nn.Softmax(dim=1)
input1 = torch.randn(2, 3)
m(input1)


tensor([[0.0946, 0.1783, 0.7271],
        [0.1455, 0.6320, 0.2226]])

In [90]:
model(X_test_tensors_final)

tensor([[0.4940],
        [0.4926],
        [0.5410],
        ...,
        [0.5135],
        [0.5998],
        [0.5796]], grad_fn=<AddmmBackward0>)

In [99]:
model(X_test_tensors_final)

tensor([[0.4940],
        [0.4926],
        [0.5410],
        ...,
        [0.5135],
        [0.5998],
        [0.5796]], grad_fn=<AddmmBackward0>)

In [ ]:
y_pred_tag =torch.round(model(X_test_tensors_final))

In [112]:
y_pred_tag

tensor([[0.],
        [0.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]], grad_fn=<RoundBackward0>)

In [113]:
y_test_tensors_final

tensor([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]])

In [114]:
y_pred_tag[0]!=y_test_tensors_final[0]

tensor([True])

In [115]:
k=0
for i in range(0,y_pred_tag.shape[0]):
    if y_pred_tag[i]!=y_test_tensors_final[i]:
        k=k+1

In [117]:
k/y_pred_tag.shape[0]

0.4072271306344392